In [1]:
import pandas as pd
import yaml
import numpy as np

In [2]:
with open('812645.yaml','r') as file:
    game = yaml.load(file)

In [3]:
game['info']

{'city': 'Abu Dhabi',
 'dates': [datetime.date(2015, 1, 14)],
 'match_type': 'ODI',
 'neutral_venue': 1,
 'outcome': {'by': {'runs': 150}, 'winner': 'Scotland'},
 'overs': 50,
 'player_of_match': ['JH Davey'],
 'teams': ['Afghanistan', 'Scotland'],
 'toss': {'decision': 'bat', 'winner': 'Scotland'},
 'umpires': ['VA Kulkarni', 'Sharfuddoula'],
 'venue': 'Sheikh Zayed Stadium'}

In [4]:
bool(game['info'].get('player_of_match'))

True

In [5]:
def get_game_info(info_dict):
    start_date = info_dict['dates'][0]
    venue = info_dict['venue']
    city = info_dict['city']
    match_type = info_dict['match_type']
    neutral = info_dict.get('neutral_venue',0)
    toss_winner = info_dict['toss']['winner']
    toss_loser = list(set(info_dict['teams']) - set([toss_winner]))[0]
    toss_decision = info_dict['toss']['decision']
    team_inn1 = toss_winner if toss_decision == 'bat' else toss_loser
    team_inn2 = list(set(info_dict['teams']) - set([team_inn1]))
    umpires = '|'.join(info_dict['umpires'])
    man_of_match = '|'.join(info_dict['player_of_match'])
    if info_dict.get('result') is not None:
        match_winner = info_dict['outcome']['by']['winner']
        win_type = info_dict['outcome']['by'].keys()[0]
        margin = info_dict['outcome']['by'][win_type]
    else:
        match_winner = info_dict['result']
        win_type = 'NA'
        margin = 'NA'
    overs = info_dict['overs']
    return (start_date, city, venue, neutral, match_type, overs, team_inn1, team_inn2, toss_winner, toss_decision, 
            match_winner, win_type, win_margin, umpires, man_of_match)

In [6]:
game['innings'][0]['1st innings']['deliveries'][32][5.3]['wicket']

KeyError: 'wicket'

In [7]:
ball = []
bowler = []
batsman = []
non_striker = []
batted_runs = []
extras = []
total_runs = []
wickets = []

In [8]:
for b in game['innings'][0]['1st innings']['deliveries']:
    ball_details = b[b.keys()[0]]
    ball.append(str(b.keys()[0]))
    bowler.append(ball_details['bowler'])
    batsman.append(ball_details['batsman'])
    non_striker.append(ball_details['non_striker'])
    batted_runs.append(ball_details['runs']['batsman'])
    extras.append(ball_details['runs']['extras'])
    total_runs.append(ball_details['runs']['total'])
    try:
        ball_details['wicket']
        wickets.append(True)
    except KeyError:
        wickets.append(False)

In [9]:
pbyp = pd.DataFrame({'ball':ball,
                     'bowler':bowler,
                     'batsman':batsman,
                     'non_striker':non_striker,
                     'batted_runs':batted_runs,
                     'extras':extras,
                     'total_runs':total_runs,
                     'wicket':wickets})

In [10]:
pbyp['innings_total'] = np.cumsum(pbyp.total_runs)
pbyp['wickets_fallen'] = np.cumsum(pbyp.wicket)

In [11]:
pbyp.head()

,ball,batsman,batted_runs,bowler,extras,non_striker,total_runs,wicket,innings_total,wickets_fallen
0,0.1,KJ Coetzer,0,Izatullah Dawlatzai,0,CS MacLeod,0,False,0,0
1,0.2,KJ Coetzer,0,Izatullah Dawlatzai,0,CS MacLeod,0,False,0,0
2,0.3,KJ Coetzer,4,Izatullah Dawlatzai,0,CS MacLeod,4,False,4,0
3,0.4,KJ Coetzer,0,Izatullah Dawlatzai,0,CS MacLeod,0,False,4,0
4,0.5,KJ Coetzer,0,Izatullah Dawlatzai,0,CS MacLeod,0,False,4,0
